In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd

# Укажите имя папки, которую вы создали на Google Диске
project_folder = 'my_project'

# Базовый путь
base_path = f'/content/drive/MyDrive/{project_folder}'

# Пути к CSV файлам (названия взяты точно с вашего скриншота)
csv_path_1 = os.path.join(base_path, 'fitzpatrick17k_ham10000_format_filled.csv')
csv_path_2 = os.path.join(base_path, 'HAM10000_metadata.csv')

# Пути к папкам с картинками
img_dir_resampled = os.path.join(base_path, 'dataset_resampled')
img_dir_fitz = os.path.join(base_path, 'fitzpatrick_dataset_images')

# --- ПРОВЕРКА ---
# Проверяем, видит ли Colab файлы
print(f"Проверяем путь: {csv_path_1}")
if os.path.exists(csv_path_1):
    print("✅ Файл CSV найден!")
    # Читаем файл для проверки
    df = pd.read_csv(csv_path_1)
    print(f"Таблица загружена. Размер: {df.shape}")
    print(df.head(3))
else:
    print("❌ Файл не найден. Проверьте, правильно ли вы указали имя папки project_folder")

Проверяем путь: /content/drive/MyDrive/my_project/fitzpatrick17k_ham10000_format_filled.csv
✅ Файл CSV найден!
Таблица загружена. Размер: (2201, 7)
      lesion_id                          image_id     dx    dx_type   age  \
0  FITZ_e783339  e783339effbc628d8b5392db27ba6716  akiec      histo  45.0   
1  FITZ_8cb31aa  8cb31aa81fca4e58aad1a07e8e9a06ce  akiec      histo  50.0   
2  FITZ_184d4c8  184d4c82d2d715ccc299db11357ea24c  akiec  consensus  45.0   

    sex localization  
0  male         back  
1  male         back  
2  male         face  


In [ ]:
import pandas as pd
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 1. Настройки
csv_file = 'fitzpatrick17k_filtered_diagnoses.csv'
output_dir = 'fitzpatrick_dataset'  # Главная папка для фото
metadata_output = 'fitzpatrick_metadata_local.csv' # Новая метадата с локальными путями

# 2. Читаем CSV
print(f"Чтение {csv_file}...")
df = pd.read_csv(csv_file)

# Создаем папки для каждого диагноза
classes = df['diagnosis_short'].unique()
print(f"Создание папок для классов: {classes}")

for cls in classes:
    os.makedirs(os.path.join(output_dir, cls), exist_ok=True)

# 3. Функция для скачивания одного изображения
def download_image(row):
    url = row.url
    label = row.diagnosis_short
    img_hash = row.md5hash

    # Путь куда сохраним: dataset/label/hash.jpg
    filename = f"{img_hash}.jpg"
    file_path = os.path.join(output_dir, label, filename)

    # Если файл уже есть, пропускаем
    if os.path.exists(file_path):
        return row.Index, file_path, True

    try:
        # Притворяемся браузером, чтобы нас не блокировали
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            return row.Index, file_path, True
        else:
            return row.Index, None, False
    except Exception as e:
        return row.Index, None, False

# 4. Запуск скачивания (в многопоточном режиме для скорости)
print("\nНачинаем скачивание изображений (это займет время)...")

# Добавляем колонку для локального пути
df['local_path'] = None
successful_downloads = 0

# Используем 10 потоков
with ThreadPoolExecutor(max_workers=10) as executor:
    # Запускаем задачи
    results = list(tqdm(executor.map(download_image, df.itertuples()), total=len(df)))

# 5. Обработка результатов
for index, path, success in results:
    if success:
        df.at[index, 'local_path'] = path
        successful_downloads += 1

print(f"\n--- Итог ---")
print(f"Всего ссылок: {len(df)}")
print(f"Успешно скачано: {successful_downloads}")
print(f"Ошибок скачивания: {len(df) - successful_downloads} (битые ссылки)")

# 6. Сохраняем обновленную метадату
# Убираем строки, где фото не скачалось
df_clean = df.dropna(subset=['local_path'])
df_clean.to_csv(metadata_output, index=False)

print(f"\nНовая метадата сохранена в: {metadata_output}")
print(f"Изображения лежат в папке: {output_dir}/")

# 7. (Опционально) Создать архив, чтобы скачать все сразу
import shutil
print("\nАрхивация изображений для скачивания...")
shutil.make_archive('fitzpatrick_dataset_images', 'zip', output_dir)
print("Архив 'fitzpatrick_dataset_images.zip' готов. Можете скачать его из панели слева.")

Чтение fitzpatrick17k_filtered_diagnoses.csv...
Создание папок для классов: ['akiec' 'bcc' 'nv' 'df' 'vasc' 'mel' 'bkl']

Начинаем скачивание изображений (это займет время)...


100%|██████████| 2201/2201 [03:07<00:00, 11.74it/s]



--- Итог ---
Всего ссылок: 2201
Успешно скачано: 2195
Ошибок скачивания: 6 (битые ссылки)

Новая метадата сохранена в: fitzpatrick_metadata_local.csv
Изображения лежат в папке: fitzpatrick_dataset/

Архивация изображений для скачивания...
Архив 'fitzpatrick_dataset_images.zip' готов. Можете скачать его из панели слева.


In [ ]:
import shutil
import os
from google.colab import files

# Имена папок и файлов
image_folder = 'fitzpatrick_dataset'       # Папка, куда скачались картинки
zip_name = 'fitzpatrick_dataset_images'    # Имя будущего архива (без .zip)
metadata_file = 'fitzpatrick_metadata_local.csv' # Файл с метаданными

# 1. Создаем архив (если папка с картинками существует)
if os.path.exists(image_folder):
    print(f"Архивирую папку '{image_folder}'... Подождите...")
    shutil.make_archive(zip_name, 'zip', image_folder)
    print(f"Архив '{zip_name}.zip' успешно создан.")
else:
    print(f"ОШИБКА: Папка '{image_folder}' не найдена. Сначала запустите код для скачивания фото.")

# 2. Скачиваем архив с фото
zip_full_name = zip_name + '.zip'
if os.path.exists(zip_full_name):
    print(f"Начинаю скачивание {zip_full_name}...")
    files.download(zip_full_name)

# 3. Скачиваем метаданные
if os.path.exists(metadata_file):
    print(f"Начинаю скачивание {metadata_file}...")
    files.download(metadata_file)

Архивирую папку 'fitzpatrick_dataset'... Подождите...
Архив 'fitzpatrick_dataset_images.zip' успешно создан.
Начинаю скачивание fitzpatrick_dataset_images.zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Начинаю скачивание fitzpatrick_metadata_local.csv...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

# 1. Загружаем оба файла
# Убедитесь, что 'HAM10000_metadata.csv' и 'fitzpatrick17k_filtered_diagnoses.csv' загружены в файлы слева
ham_df = pd.read_csv('HAM10000_metadata.csv')
fitz_df = pd.read_csv('fitzpatrick17k_filtered_diagnoses.csv')

# 2. Создаем новую структуру
new_metadata = pd.DataFrame()
new_metadata['lesion_id'] = 'FITZ_' + fitz_df['md5hash'].str[:7]
new_metadata['image_id'] = fitz_df['md5hash']
new_metadata['dx'] = fitz_df['diagnosis_short']

# 3. Список колонок, которые нужно заполнить рандомно
columns_to_fill = ['dx_type', 'age', 'sex', 'localization']

print("Заполняем пропуски на основе статистики HAM10000...")

for col in columns_to_fill:
    # Считаем вероятность появления каждого значения в HAM10000
    # (normalize=True дает проценты, dropna=True убирает пустые значения из расчета)
    value_counts = ham_df[col].value_counts(normalize=True, dropna=True)

    values = value_counts.index.tolist()       # Варианты (например, 'male', 'female')
    probabilities = value_counts.values.tolist() # Их вероятности (например, 0.54, 0.46)

    # Генерируем случайный список для нашего нового датасета с учетом этих вероятностей
    random_values = np.random.choice(values, size=len(new_metadata), p=probabilities)

    # Записываем в колонку
    new_metadata[col] = random_values

# 4. Упорядочиваем колонки
target_columns = ['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization']
new_metadata = new_metadata[target_columns]

# 5. Сохраняем и скачиваем
output_file = 'fitzpatrick17k_ham10000_format_filled.csv'
new_metadata.to_csv(output_file, index=False)

print(f"\nФайл {output_file} успешно создан!")
print(new_metadata.head())

files.download(output_file)

Заполняем пропуски на основе статистики HAM10000...

Файл fitzpatrick17k_ham10000_format_filled.csv успешно создан!
      lesion_id                          image_id     dx    dx_type   age  \
0  FITZ_e783339  e783339effbc628d8b5392db27ba6716  akiec      histo  45.0   
1  FITZ_8cb31aa  8cb31aa81fca4e58aad1a07e8e9a06ce  akiec      histo  50.0   
2  FITZ_184d4c8  184d4c82d2d715ccc299db11357ea24c  akiec  consensus  45.0   
3  FITZ_f5d4119  f5d4119aa4334fce797396a081e0686d  akiec      histo  60.0   
4  FITZ_6271153  62711536b80f9a897c6d5bde4f320397  akiec  follow_up  80.0   

    sex localization  
0  male         back  
1  male         back  
2  male         face  
3  male         back  
4  male      abdomen  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ВНИМАНИЕ: Это удалит папку безвозвратно!
# Замените путь на свой

!rm -rf local_ham

In [ ]:
import os
import shutil
from google.colab import drive

# 1. Подключаем диск (если уже подключен - ничего страшного, сообщение про remount игнорируем)
drive.mount('/content/drive')

# ================= НАСТРОЙКИ =================
# Пути к исходным файлам на Google Drive
DRIVE_HAM_DIR = '/content/drive/MyDrive/my_project/dataset_resampled'
DRIVE_FITZ_DIR = '/content/drive/MyDrive/my_project/fitzpatrick_dataset'

# Куда копируем (локально в Colab)
LOCAL_HAM_DIR = '/content/local_ham'
LOCAL_FITZ_DIR = '/content/local_fitz'

# ================= ФУНКЦИЯ ПЕРЕЗАПУСКА =================
def clean_and_copy(src_path, dst_path):
    # 1. Удаляем старую папку, если она есть (чтобы убрать ошибки)
    if os.path.exists(dst_path):
        print(f"🧹 Удаляю старую/поврежденную папку: {dst_path}...")
        shutil.rmtree(dst_path) # Полное удаление папки со всем содержимым

    # 2. Копируем заново
    print(f"🚀 Копирую {src_path} -> {dst_path}...")
    try:
        # shutil.copytree создает папку назначения и копирует всё дерево файлов
        shutil.copytree(src_path, dst_path)
        print(f"✅ Успешно скопировано: {dst_path}")
    except Exception as e:
        print(f"❌ Ошибка при копировании: {e}")

# ================= ЗАПУСК =================

# Исправляем HAM10000
clean_and_copy(DRIVE_HAM_DIR, LOCAL_HAM_DIR)

# Исправляем Fitzpatrick (если нужно)
clean_and_copy(DRIVE_FITZ_DIR, LOCAL_FITZ_DIR)

print("\n🏁 Все готово! Теперь можно запускать код обучения.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Копирую /content/drive/MyDrive/my_project/dataset_resampled -> /content/local_ham...
✅ Успешно скопировано: /content/local_ham
🚀 Копирую /content/drive/MyDrive/my_project/fitzpatrick_dataset -> /content/local_fitz...
✅ Успешно скопировано: /content/local_fitz

🏁 Все готово! Теперь можно запускать код обучения.


In [ ]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# 1. Настройки (используем ЛОКАЛЬНЫЕ пути из прошлого шага)
HAM_DIR = '/content/local_ham'
FITZ_DIR = '/content/local_fitz'

# CSV читаем прямо с диска (они маленькие, это быстро)
HAM_CSV = os.path.join(PROJECT_PATH, 'HAM10000_metadata.csv')
FITZ_CSV = os.path.join(PROJECT_PATH, 'fitzpatrick17k_ham10000_format_filled.csv')

IMG_SIZE = 300
BATCH_SIZE = 32
EPOCHS = 25

# 2. Загрузка и подготовка метаданных
print("Чтение метаданных...")
df_ham = pd.read_csv(HAM_CSV)
df_fitz = pd.read_csv(FITZ_CSV)

# Функция поиска файлов в локальных папках
def find_path(image_id, root_dir):
    # Быстрый путь (если файл лежит в корне папки)
    path = os.path.join(root_dir, f"{image_id}.jpg")
    if os.path.exists(path): return path

    # Если не нашли, ищем во вложенных папках (рекурсивно)
    for root, _, files in os.walk(root_dir):
        if f"{image_id}.jpg" in files:
            return os.path.join(root, f"{image_id}.jpg")
    return None

print("Привязка путей к файлам (это быстро)...")
# Ищем картинки HAM в локальной папке
df_ham['path'] = df_ham['image_id'].apply(lambda x: find_path(x, HAM_DIR))
# Ищем картинки Fitz в локальной папке
df_fitz['path'] = df_fitz.apply(lambda row: find_path(row['image_id'], os.path.join(FITZ_DIR, row['dx'])), axis=1)
# (Для Fitz мы знаем структуру папок /dx/, поэтому ищем сразу там для скорости, или fallback find_path)
# Упростим для надежности:
df_fitz['path'] = df_fitz['image_id'].apply(lambda x: find_path(x, FITZ_DIR))


# Удаляем пустые
df_ham = df_ham.dropna(subset=['path'])
df_fitz = df_fitz.dropna(subset=['path'])

# Объединяем
full_df = pd.concat([df_ham, df_fitz], ignore_index=True)
print(f"\nВсего изображений найдено: {len(full_df)}")
print(full_df['dx'].value_counts())

# Разделение
train_df, val_df = train_test_split(full_df, test_size=0.2, stratify=full_df['dx'], random_state=42)

# 3. Генераторы
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='path', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='path', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# 4. Веса классов
weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = dict(enumerate(weights))

# 5. Модель EfficientNetV2
print("\nЗагрузка EfficientNetV2...")
base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), include_preprocessing=True)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
outputs = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Обучение
print("\nНачинаем обучение (Head)...")
checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(PROJECT_PATH, 'best_skin_model.h5'), monitor='val_accuracy', save_best_only=True, verbose=1)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=[checkpoint]
)

# ================= 7. УЛУЧШЕННЫЙ FINE-TUNING =================
print("\nНачинаем ГЛУБОКИЙ Fine-Tuning...")

# Размораживаем ВСЮ модель (рискованно, но эффективно при малом LR)
base_model.trainable = True

# Критически важно: ОЧЕНЬ маленький LR
model.compile(
    optimizer=Adam(learning_rate=1e-6),  # Было 1e-5, ставим еще меньше
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Увеличиваем количество эпох для тонкой настройки
history_fine = model.fit(
    train_generator,
    epochs=15,  # Даем ей время медленно перестроиться
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=[checkpoint, reduce_lr, early_stop]
)

print(f"\nГотово! Модель сохранена на Google Drive: {os.path.join(PROJECT_PATH, 'best_skin_model.h5')}")

Чтение метаданных...
Привязка путей к файлам (это быстро)...

Всего изображений найдено: 2816
dx
bcc      696
mel      670
vasc     538
nv       333
akiec    275
bkl      169
df       135
Name: count, dtype: int64
Found 2252 validated image filenames belonging to 7 classes.
Found 564 validated image filenames belonging to 7 classes.

Загрузка EfficientNetV2...

Начинаем обучение (Head)...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 956ms/step - accuracy: 0.2052 - loss: 2.5748
Epoch 1: val_accuracy improved from -inf to 0.41667, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.2060 - loss: 2.5713 - val_accuracy: 0.4167 - val_loss: 1.5882
Epoch 2/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 770ms/step - accuracy: 0.3621 - loss: 1.8507
Epoch 2: val_accuracy improved from 0.41667 to 0.49823, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 60s 843ms/step - accuracy: 0.3620 - loss: 1.8512 - val_accuracy: 0.4982 - val_loss: 1.4273
Epoch 3/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 766ms/step - accuracy: 0.3833 - loss: 1.7738
Epoch 3: val_accuracy improved from 0.49823 to 0.50532, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


...

71/71 ━━━━━━━━━━━━━━━━━━━━ 58s 821ms/step - accuracy: 0.4556 - loss: 1.5130 - val_accuracy: 0.5319 - val_loss: 1.3347
Epoch 6/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - accuracy: 0.5013 - loss: 1.3701
Epoch 6: val_accuracy did not improve from 0.53191
71/71 ━━━━━━━━━━━━━━━━━━━━ 58s 822ms/step - accuracy: 0.5011 - loss: 1.3707 - val_accuracy: 0.4858 - val_loss: 1.4618
Epoch 7/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 758ms/step - accuracy: 0.5121 - loss: 1.3893
Epoch 7: val_accuracy improved from 0.53191 to 0.54078, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 59s 828ms/step - accuracy: 0.5118 - loss: 1.3893 - val_accuracy: 0.5408 - val_loss: 1.3484
Epoch 8/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 762ms/step - accuracy: 0.5176 - loss: 1.2620
Epoch 8: val_accuracy did not improve from 0.54078
71/71 ━━━━━━━━━━━━━━━━━━━━ 57s 809ms/step - accuracy: 0.5175 - loss: 1.2631 - val_accuracy: 0.5053 - val_loss: 1.4075
Epoch 9/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 759ms/step - accuracy: 0.5235 - loss: 1.3194
Epoch 9: val_accuracy did not improve from 0.54078
71/71 ━━━━━━━━━━━━━━━━━━━━ 58s 808ms/step - accuracy: 0.5234 - loss: 1.3200 - val_accuracy: 0.5018 - val_loss: 1.4808
Epoch 10/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 759ms/step - accuracy: 0.5280 - loss: 1.2642
Epoch 10: val_accuracy did not improve from 0.54078
71/71 ━━━━━━━━━━━━━━━━━━━━ 57s 803ms/step - accuracy: 0.5279 - loss: 1.2649 - val_accuracy: 0.5195 - val_loss: 1.3293
Epoch 11/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 756ms/step - accuracy: 0.5522 - loss: 1.3002
Epoch 11: val_accuracy impro

71/71 ━━━━━━━━━━━━━━━━━━━━ 59s 837ms/step - accuracy: 0.5521 - loss: 1.3001 - val_accuracy: 0.5479 - val_loss: 1.2856
Epoch 12/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 756ms/step - accuracy: 0.5548 - loss: 1.2262
Epoch 12: val_accuracy did not improve from 0.54787
71/71 ━━━━━━━━━━━━━━━━━━━━ 57s 800ms/step - accuracy: 0.5547 - loss: 1.2265 - val_accuracy: 0.5319 - val_loss: 1.3471
Epoch 13/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 748ms/step - accuracy: 0.5384 - loss: 1.2557
Epoch 13: val_accuracy did not improve from 0.54787
71/71 ━━━━━━━━━━━━━━━━━━━━ 56s 792ms/step - accuracy: 0.5383 - loss: 1.2559 - val_accuracy: 0.5284 - val_loss: 1.3493
Epoch 14/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 752ms/step - accuracy: 0.5339 - loss: 1.2735
Epoch 14: val_accuracy improved from 0.54787 to 0.56383, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 59s 826ms/step - accuracy: 0.5340 - loss: 1.2735 - val_accuracy: 0.5638 - val_loss: 1.3280
Epoch 15/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 757ms/step - accuracy: 0.5466 - loss: 1.2423
Epoch 15: val_accuracy did not improve from 0.56383
71/71 ━━━━━━━━━━━━━━━━━━━━ 57s 800ms/step - accuracy: 0.5467 - loss: 1.2420 - val_accuracy: 0.5550 - val_loss: 1.3597
Epoch 16/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 754ms/step - accuracy: 0.5302 - loss: 1.2412
Epoch 16: val_accuracy did not improve from 0.56383
71/71 ━━━━━━━━━━━━━━━━━━━━ 56s 797ms/step - accuracy: 0.5302 - loss: 1.2411 - val_accuracy: 0.5408 - val_loss: 1.3375
Epoch 17/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 749ms/step - accuracy: 0.5521 - loss: 1.1608
Epoch 17: val_accuracy did not improve from 0.56383
71/71 ━━━━━━━━━━━━━━━━━━━━ 57s 800ms/step - accuracy: 0.5521 - loss: 1.1610 - val_accuracy: 0.5230 - val_loss: 1.4741
Epoch 18/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 758ms/step - accuracy: 0.5451 - loss: 1.2175
Epoch 18: val_accuracy d

71/71 ━━━━━━━━━━━━━━━━━━━━ 60s 849ms/step - accuracy: 0.5510 - loss: 1.1931 - val_accuracy: 0.5691 - val_loss: 1.2589
Epoch 24/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 757ms/step - accuracy: 0.5618 - loss: 1.1238
Epoch 24: val_accuracy did not improve from 0.56915
71/71 ━━━━━━━━━━━━━━━━━━━━ 57s 801ms/step - accuracy: 0.5617 - loss: 1.1243 - val_accuracy: 0.5461 - val_loss: 1.3834
Epoch 25/25
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 752ms/step - accuracy: 0.5857 - loss: 1.1062
Epoch 25: val_accuracy did not improve from 0.56915
71/71 ━━━━━━━━━━━━━━━━━━━━ 56s 794ms/step - accuracy: 0.5856 - loss: 1.1066 - val_accuracy: 0.5443 - val_loss: 1.3226

Начинаем ГЛУБОКИЙ Fine-Tuning...
Epoch 1/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3175 - loss: 2.2523
Epoch 1: val_accuracy did not improve from 0.56915
71/71 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.3178 - loss: 2.2508 - val_accuracy: 0.3014 - val_loss: 2.0888 - learning_rate: 1.0000e-06
Epoch 2/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 814ms/step - ac

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# 1. Настройки
checkpoint_path = '/content/drive/MyDrive/my_project/best_skin_model.h5'

# Загружаем модель (которую вы только что переобучили на 1 этапе)
print(f"Загружаю веса после 1 этапа: {checkpoint_path}")
model.load_weights(checkpoint_path)

# =================================================================
# ГЛАВНОЕ ИСПРАВЛЕНИЕ: "Умная" разморозка
# =================================================================
base_model.trainable = True

# Проходим по всем слоям и замораживаем ТОЛЬКО BatchNormalization
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

print("✅ Модель разморожена, но слои BatchNormalization защищены!")
# =================================================================

# 2. Компилируем с низким LR
model.compile(
    optimizer=Adam(learning_rate=1e-6), # Очень маленький шаг
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)

# 3. Запускаем Fine-Tuning
print("\n🚀 Запуск ИСПРАВЛЕННОГО Fine-Tuning...")
history_fine = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=[checkpoint, reduce_lr, early_stop]
)

Загружаю веса после 1 этапа: /content/drive/MyDrive/my_project/best_skin_model.h5
✅ Модель разморожена, но слои BatchNormalization защищены!

🚀 Запуск ИСПРАВЛЕННОГО Fine-Tuning...
Epoch 1/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5841 - loss: 1.0519
Epoch 1: val_accuracy improved from -inf to 0.57270, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.5840 - loss: 1.0525 - val_accuracy: 0.5727 - val_loss: 1.2646 - learning_rate: 1.0000e-06
Epoch 2/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 818ms/step - accuracy: 0.5912 - loss: 1.0803
Epoch 2: val_accuracy improved from 0.57270 to 0.57801, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 65s 914ms/step - accuracy: 0.5912 - loss: 1.0805 - val_accuracy: 0.5780 - val_loss: 1.2580 - learning_rate: 1.0000e-06
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 812ms/step - accuracy: 0.5903 - loss: 1.0421
Epoch 3: val_accuracy did not improve from 0.57801
71/71 ━━━━━━━━━━━━━━━━━━━━ 61s 857ms/step - accuracy: 0.5903 - loss: 1.0426 - val_accuracy: 0.5745 - val_loss: 1.2549 - learning_rate: 1.0000e-06
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 792ms/step - accuracy: 0.5825 - loss: 1.1004
Epoch 4: val_accuracy improved from 0.57801 to 0.57979, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 63s 874ms/step - accuracy: 0.5825 - loss: 1.1002 - val_accuracy: 0.5798 - val_loss: 1.2509 - learning_rate: 1.0000e-06
Epoch 5/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 836ms/step - accuracy: 0.5563 - loss: 1.1456
Epoch 5: val_accuracy did not improve from 0.57979
71/71 ━━━━━━━━━━━━━━━━━━━━ 62s 879ms/step - accuracy: 0.5565 - loss: 1.1451 - val_accuracy: 0.5780 - val_loss: 1.2511 - learning_rate: 1.0000e-06
Epoch 6/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 788ms/step - accuracy: 0.5806 - loss: 1.1049
Epoch 6: val_accuracy did not improve from 0.57979
71/71 ━━━━━━━━━━━━━━━━━━━━ 59s 832ms/step - accuracy: 0.5806 - loss: 1.1050 - val_accuracy: 0.5798 - val_loss: 1.2458 - learning_rate: 1.0000e-06
Epoch 7/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - accuracy: 0.5941 - loss: 1.0665
Epoch 7: val_accuracy did not improve from 0.57979
71/71 ━━━━━━━━━━━━━━━━━━━━ 61s 850ms/step - accuracy: 0.5941 - loss: 1.0666 - val_accuracy: 0.5798 - val_loss: 1.2443 - learning_rate: 1.0000e-06
Epoch

71/71 ━━━━━━━━━━━━━━━━━━━━ 66s 930ms/step - accuracy: 0.5885 - loss: 1.0796 - val_accuracy: 0.5833 - val_loss: 1.2456 - learning_rate: 1.0000e-06
Epoch 9/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 805ms/step - accuracy: 0.5870 - loss: 1.1026
Epoch 9: val_accuracy improved from 0.58333 to 0.58688, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


...

71/71 ━━━━━━━━━━━━━━━━━━━━ 64s 899ms/step - accuracy: 0.5941 - loss: 1.0494 - val_accuracy: 0.5957 - val_loss: 1.2424 - learning_rate: 1.0000e-06
Epoch 12/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 814ms/step - accuracy: 0.6067 - loss: 1.0722
Epoch 12: val_accuracy did not improve from 0.59574
71/71 ━━━━━━━━━━━━━━━━━━━━ 63s 887ms/step - accuracy: 0.6066 - loss: 1.0722 - val_accuracy: 0.5957 - val_loss: 1.2422 - learning_rate: 1.0000e-06
Epoch 13/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 794ms/step - accuracy: 0.6080 - loss: 1.0382
Epoch 13: val_accuracy improved from 0.59574 to 0.60106, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 62s 874ms/step - accuracy: 0.6079 - loss: 1.0383 - val_accuracy: 0.6011 - val_loss: 1.2378 - learning_rate: 1.0000e-06
Epoch 14/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 826ms/step - accuracy: 0.6021 - loss: 1.0152
Epoch 14: val_accuracy did not improve from 0.60106
71/71 ━━━━━━━━━━━━━━━━━━━━ 62s 868ms/step - accuracy: 0.6019 - loss: 1.0159 - val_accuracy: 0.5975 - val_loss: 1.2376 - learning_rate: 1.0000e-06
Epoch 15/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 787ms/step - accuracy: 0.5746 - loss: 1.1104
Epoch 15: val_accuracy did not improve from 0.60106
71/71 ━━━━━━━━━━━━━━━━━━━━ 59s 834ms/step - accuracy: 0.5747 - loss: 1.1102 - val_accuracy: 0.5993 - val_loss: 1.2344 - learning_rate: 1.0000e-06
Epoch 16/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 798ms/step - accuracy: 0.6057 - loss: 1.0135
Epoch 16: val_accuracy improved from 0.60106 to 0.60284, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 63s 879ms/step - accuracy: 0.6056 - loss: 1.0136 - val_accuracy: 0.6028 - val_loss: 1.2372 - learning_rate: 1.0000e-06
Epoch 17/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 826ms/step - accuracy: 0.6117 - loss: 1.0082
Epoch 17: val_accuracy did not improve from 0.60284
71/71 ━━━━━━━━━━━━━━━━━━━━ 62s 869ms/step - accuracy: 0.6117 - loss: 1.0079 - val_accuracy: 0.6028 - val_loss: 1.2336 - learning_rate: 1.0000e-06
Epoch 18/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 791ms/step - accuracy: 0.6016 - loss: 1.0442
Epoch 18: val_accuracy did not improve from 0.60284
71/71 ━━━━━━━━━━━━━━━━━━━━ 59s 833ms/step - accuracy: 0.6017 - loss: 1.0438 - val_accuracy: 0.6011 - val_loss: 1.2308 - learning_rate: 1.0000e-06
Epoch 19/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 807ms/step - accuracy: 0.6085 - loss: 0.9977
Epoch 19: val_accuracy improved from 0.60284 to 0.60638, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 65s 916ms/step - accuracy: 0.6085 - loss: 0.9979 - val_accuracy: 0.6064 - val_loss: 1.2306 - learning_rate: 1.0000e-06
Epoch 20/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 831ms/step - accuracy: 0.6121 - loss: 1.0206
Epoch 20: val_accuracy improved from 0.60638 to 0.60993, saving model to /content/drive/MyDrive/my_project/best_skin_model.h5


71/71 ━━━━━━━━━━━━━━━━━━━━ 67s 951ms/step - accuracy: 0.6121 - loss: 1.0202 - val_accuracy: 0.6099 - val_loss: 1.2269 - learning_rate: 1.0000e-06
Restoring model weights from the end of the best epoch: 20.
